In [ ]:
import pandas as pd
import numpy as np
from os import path

In [15]:
weather_train = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')
weather_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')


In [16]:
def fancyBackfilling(df):
    #do fancy stuff
    return df

In [26]:
# https://www.kaggle.com/nz0722/aligned-timestamp-lgbm-by-meter-type
def alignTimePickle():
    weather = pd.concat([weather_train,weather_test],ignore_index=True)
    weather['timestamp'] = pd.to_datetime(weather['timestamp'])
    weather_key = ['site_id', 'timestamp']

    temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

    # calculate ranks of hourly temperatures within date/site_id chunks
    temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')
    
    # create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
    df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)
    print(df_2d)
    # Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
    site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
    site_ids_offsets.index.name = 'site_id'
    site_ids_offsets.to_pickle('../input/ashrae-energy-prediction/site_time_offsets_df.pickle')
    print(site_ids_offsets)

alignTimePickle()

timestamp         0          1          2          3          4          5   \
site_id                                                                       
0          15.123175  13.351277  11.868157  10.541971   9.299270   7.907391   
1           8.950691   7.786636   6.781049   5.786175   5.062557   4.993560   
2          21.754562  20.260493  18.296804  16.325730  14.544252  12.886405   
3          16.497715  14.732176  13.162409  11.897717  10.696347   9.474406   
4          20.952425  19.079836  16.622719  14.306569  12.331661  10.831661   
5           9.062095   8.491220   7.997692   7.461716   6.926199   6.691705   
6          16.386093  14.572536  13.068431  11.704296  10.390768   9.168037   
7          16.195570  14.469885  12.982364  11.843602  10.484630   9.453200   
8          15.123175  13.351277  11.868157  10.541971   9.299270   7.907391   
9          18.589450  16.621448  14.826782  13.238792  11.761208  10.164070   
10         19.833943  18.026916  15.886405  13.72872

In [17]:
weather_train = fancyBackfilling(weather_train);
weather_test = fancyBackfilling(weather_test);

In [18]:
# savve modified files in _ directory
if not path.exists('../input/_ashrae-energy-prediction'):
     !mkdir -p ../input/_ashrae-energy-prediction
    
weather_train.to_csv('../input/_ashrae-energy-prediction/weather_train.csv')
weather_test.to_csv('../input/_ashrae-energy-prediction/weather_test.csv')
